In [1]:
import pandas as pd

In [5]:
url = f'https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/04-monitoring/data/results-gpt4o-mini.csv?raw=1'
df = pd.read_csv(url)

In [6]:

df = df.iloc[:300]

In [7]:
df.head()

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp


In [8]:
from sentence_transformers import SentenceTransformer

# Load the specified model
model_name = 'multi-qa-mpnet-base-dot-v1'
embedding_model = SentenceTransformer(model_name)

# Assuming you have a DataFrame 'df' with a column 'answer_llm'
answer_llm = df.iloc[0].answer_llm

# Create the embeddings
embeddings = embedding_model.encode(answer_llm)

# Check the first value of the resulting vector
first_value = embeddings[0]

print(f"The first value of the resulting vector is: {first_value}")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

You try to use a model that was created with version 3.0.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

The first value of the resulting vector is: -0.4224465489387512


In [11]:
import numpy as np
from sentence_transformers import SentenceTransformer

# Load the model
model_name = 'multi-qa-mpnet-base-dot-v1'
embedding_model = SentenceTransformer(model_name)

# Initialize an empty list to store the scores
evaluations = []

# Iterate through each row in the dataframe
for _, row in df.iterrows():
    # Create embeddings for both answers
    embedding1 = embedding_model.encode(row['answer_llm'])
    embedding2 = embedding_model.encode(row['answer_orig'])
    
    # Compute dot product
    score = np.dot(embedding1, embedding2)
    
    # Append the score to the evaluations list
    evaluations.append(score)

# Calculate the 75th percentile
percentile_75 = np.percentile(evaluations, 75)

print(f"The 75th percentile of the scores is: {percentile_75}")

You try to use a model that was created with version 3.0.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





The 75th percentile of the scores is: 31.67430877685547


In [12]:
import numpy as np
from sentence_transformers import SentenceTransformer

# Load the model
model_name = 'multi-qa-mpnet-base-dot-v1'
embedding_model = SentenceTransformer(model_name)

# Initialize an empty list to store the scores
evaluations = []

# Function to normalize a vector
def normalize(v):
    norm = np.sqrt((v * v).sum())
    return v / norm

# Iterate through each row in the dataframe
for _, row in df.iterrows():
    # Create embeddings for both answers
    embedding1 = embedding_model.encode(row['answer_llm'])
    embedding2 = embedding_model.encode(row['answer_orig'])
    
    # Normalize the embeddings
    embedding1_norm = normalize(embedding1)
    embedding2_norm = normalize(embedding2)
    
    # Compute cosine similarity (dot product of normalized vectors)
    score = np.dot(embedding1_norm, embedding2_norm)
    
    # Append the score to the evaluations list
    evaluations.append(score)

# Calculate the 75th percentile
percentile_75 = np.percentile(evaluations, 75)

print(f"The 75th percentile of the cosine similarity scores is: {percentile_75}")

You try to use a model that was created with version 3.0.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





The 75th percentile of the cosine similarity scores is: 0.8362348973751068


In [13]:
pip install rouge -q

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [14]:
from rouge import Rouge

# Initialize the Rouge scorer
rouge_scorer = Rouge()

# Get the row at index 10 (document ID 5170565b)
r = df.iloc[10]

# Compute ROUGE scores
scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]

# Extract the F1 score for ROUGE-1
rouge_1_f1 = scores['rouge-1']['f']

print(f"The F1 score for ROUGE-1 is: {rouge_1_f1}")

The F1 score for ROUGE-1 is: 0.45454544954545456


In [15]:
from rouge import Rouge

# Initialize the Rouge scorer
rouge_scorer = Rouge()

# Get the row at index 10 (document ID 5170565b)
r = df.iloc[10]

# Compute ROUGE scores
scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]

# Extract F1 scores for ROUGE-1, ROUGE-2, and ROUGE-L
rouge_1_f1 = scores['rouge-1']['f']
rouge_2_f1 = scores['rouge-2']['f']
rouge_l_f1 = scores['rouge-l']['f']

# Calculate the average F1 score
average_f1 = (rouge_1_f1 + rouge_2_f1 + rouge_l_f1) / 3

print(f"ROUGE-1 F1: {rouge_1_f1}")
print(f"ROUGE-2 F1: {rouge_2_f1}")
print(f"ROUGE-L F1: {rouge_l_f1}")
print(f"The average F1 score across ROUGE-1, ROUGE-2, and ROUGE-L is: {average_f1}")

ROUGE-1 F1: 0.45454544954545456
ROUGE-2 F1: 0.21621621121621637
ROUGE-L F1: 0.393939388939394
The average F1 score across ROUGE-1, ROUGE-2, and ROUGE-L is: 0.35490034990035496


In [16]:
import pandas as pd
from rouge import Rouge

# Initialize the Rouge scorer
rouge_scorer = Rouge()

# Function to compute ROUGE scores for a single row
def compute_rouge_scores(row):
    scores = rouge_scorer.get_scores(row['answer_llm'], row['answer_orig'])[0]
    return {
        'rouge_1': scores['rouge-1']['f'],
        'rouge_2': scores['rouge-2']['f'],
        'rouge_l': scores['rouge-l']['f']
    }

# Compute ROUGE scores for all rows
rouge_scores = df.apply(compute_rouge_scores, axis=1)

# Convert the results to a DataFrame
rouge_df = pd.DataFrame(rouge_scores.tolist())

# Calculate the average ROUGE-2 score
average_rouge_2 = rouge_df['rouge_2'].mean()

print(f"The average ROUGE-2 score across all records is: {average_rouge_2}")

The average ROUGE-2 score across all records is: 0.20696501983423318
